# Test data loading for time-series dataset

This is a short test to show how data loading works 

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

# all paths relative to afa directory 
sys.path.insert(0, os.path.abspath('../../afa'))

### Load time-series dataset

In [2]:
from afa.data_modelling.datasets.data_loader.data_loader_ts import DataLoader_ts

In [3]:
data_dir = '../../../data/ts/synthetic/'
data_file          = data_dir + 'synthetic_static_train_data.csv.gz' 
temporal_data_file = data_dir + 'synthetic_ts_train_data_eav.csv.gz' 
# superfeature_mapping_file = data_dir + 'superfeature_names.csv'
superfeature_mapping_file = None
    
data_loader_train = DataLoader_ts( data_file                  = data_file,
                                   temporal_data_file         = temporal_data_file,
                                   superfeature_mapping_file  = superfeature_mapping_file )

dataset = data_loader_train.load() 

## Preprocessing

In [4]:
from afa.data_modelling.preprocessing import Normalizer
from afa.data_modelling.utils.model_utils import PipelineComposer

2022-12-28 11:10:08.721585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 11:10:08.844811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-28 11:10:08.844829: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-28 11:10:09.568608: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [5]:
normalization = 'minmax'
normalizer = Normalizer(normalization)

# Data preprocessing
filter_pipeline = PipelineComposer(normalizer)

dataset = filter_pipeline.fit_transform(dataset)

print('Finish preprocessing.')

Finish preprocessing.


### Define problem

Set the prediction problem that we want to solve. Set the problem (one-shot), set the label and set the treatment features (not supported). 

We also define the metric for evaluation and the task itself (whether classification or regression
  - problem: 
    - 'one-shot': one time prediction 
    - 'online': predict time-series 
  - label_name: the column name for the label(s)
  - treatment: the column name for treatments

In [6]:
from afa.data_modelling.problem.problem_ts import ProblemMaker_ts

In [7]:
label_name = 'Y_ts'
problem = 'online'
treatment = None
max_seq_len = 10
# Define problem 
problem_maker = ProblemMaker_ts(    problem    = problem, 
                                    label      = [label_name], 
                                    treatment  = treatment, 
                                    max_seq_len = max_seq_len)

dataset  = problem_maker.fit_transform(dataset)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 596.62it/s]


KeyError: 'temporal_data'

## Impute dataset

In [24]:
from clairvoyance_static.imputation import Imputation

# Set imputation models
imputation_model = 'median'

# Impute the missing data
imputation          = Imputation(imputation_model_name = imputation_model)

imputation_pipeline = PipelineComposer(imputation)

dataset_train = imputation_pipeline.fit_transform(dataset_train)
dataset_test  = imputation_pipeline.transform(dataset_test)

print('Finish imputation.')

ModuleNotFoundError: No module named 'clairvoyance_static'